### Importing required libraries

In [13]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.utils import Sequence

### Model Architecture

In [14]:
def build_srcnn_model():
    model = Sequential()
    model.add(Conv2D(64, (9, 9), activation='relu', padding='same', input_shape=(None, None, 3)))
    model.add(Conv2D(32, (1, 1), activation='relu', padding='same'))
    model.add(Conv2D(3, (5, 5), activation='linear', padding='same'))
    return model

### Compiling the Model

In [15]:
model = build_srcnn_model()
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

### Setting directories for data

In [16]:
lr_dir = "DIV2K/train_LR_X4/DIV2K_train_LR_bicubic/X4"
hr_dir = "DIV2K/train_HR/DIV2K_train_HR"

### Generate Data in patches of size 64x64 for processing

In [ ]:
class SuperResolutionDataGen(Sequence):
    def __init__(self, lr_dir, hr_dir, batch_size=8, patch_size=64):
        self.lr_files = sorted(os.listdir(lr_dir))
        self.hr_files = sorted(os.listdir(hr_dir))
        self.lr_dir = lr_dir
        self.hr_dir = hr_dir
        self.batch_size = batch_size
        self.patch_size = patch_size

    def __len__(self):
        return len(self.lr_files) // self.batch_size

    def __getitem__(self, idx):
        batch_lr = []
        batch_hr = []

        for i in range(self.batch_size):
            lr_path = os.path.join(self.lr_dir, self.lr_files[idx * self.batch_size + i])
            hr_path = os.path.join(self.hr_dir, self.hr_files[idx * self.batch_size + i])
            
            lr = cv2.imread(lr_path)
            hr = cv2.imread(hr_path)

            h, w, _ = lr.shape
            x = np.random.randint(0, w - self.patch_size)
            y = np.random.randint(0, h - self.patch_size)

            lr_patch_raw = lr[y:y+self.patch_size, x:x+self.patch_size]
            lr_patch = cv2.resize(lr_patch_raw, (self.patch_size * 4, self.patch_size * 4), interpolation=cv2.INTER_CUBIC)

            hr_patch = hr[y*4:y*4 + self.patch_size*4, x*4:x*4 + self.patch_size*4]

            # Normalize
            lr_patch = lr_patch.astype('float32') / 255.
            hr_patch = hr_patch.astype('float32') / 255.

            batch_lr.append(lr_patch)
            batch_hr.append(hr_patch)

        return np.array(batch_lr), np.array(batch_hr)

### Saving the Model

In [ ]:
train_gen = SuperResolutionDataGen(
    lr_dir=lr_dir,
    hr_dir=hr_dir,
    batch_size=4,
    patch_size=64
)
model.fit(train_gen, epochs=10)
model.save("srcnn_custom_trained.h5")
model.save('my_model.keras')

Epoch 1/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 72s 354ms/step - loss: 0.0338 - mae: 0.1121
Epoch 2/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 55s 277ms/step - loss: 0.0047 - mae: 0.0408
Epoch 3/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 61s 307ms/step - loss: 0.0047 - mae: 0.0409
Epoch 4/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 64s 319ms/step - loss: 0.0038 - mae: 0.0375
Epoch 5/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 62s 309ms/step - loss: 0.0038 - mae: 0.0348
Epoch 6/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 54s 272ms/step - loss: 0.0034 - mae: 0.0325
Epoch 7/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 57s 286ms/step - loss: 0.0039 - mae: 0.0383
Epoch 8/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 57s 285ms/step - loss: 0.0032 - mae: 0.0314
Epoch 9/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 57s 284ms/step - loss: 0.0041 - mae: 0.0397
Epoch 10/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 59s 295ms/step - loss: 0.0034 - mae: 0.0333


### Model Architecture

In [2]:
model = load_model("my_model.keras")
model.summary()

c:\Users\avixa\anaconda3\envs\ML\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, None, None, 64) │        15,616 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, None, None, 32) │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, None, None, 3)  │         2,403 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 40,200 (157.04 KB)

 Trainable params: 20,099 (78.51 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 20,101 (78.52 KB)

### Generate Data without patching

In [21]:
class FullImageValidationGen(tf.keras.utils.Sequence):
    def __init__(self, lr_dir, hr_dir):
        self.lr_files = sorted(os.listdir(lr_dir))
        self.hr_files = sorted(os.listdir(hr_dir))
        self.lr_dir = lr_dir
        self.hr_dir = hr_dir

    def __len__(self):
        return len(self.lr_files)

    def __getitem__(self, idx):
        lr_path = os.path.join(self.lr_dir, self.lr_files[idx])
        hr_path = os.path.join(self.hr_dir, self.hr_files[idx])

        # Load images
        lr = cv2.imread(lr_path)
        hr = cv2.imread(hr_path)

        if lr is None or hr is None:
            raise ValueError(f"Missing file: {lr_path} or {hr_path}")

        lr = cv2.cvtColor(lr, cv2.COLOR_BGR2RGB)
        hr = cv2.cvtColor(hr, cv2.COLOR_BGR2RGB)

        # Upsample LR to HR size using bicubic
        h, w, _ = hr.shape
        lr_upscaled = cv2.resize(lr, (w, h), interpolation=cv2.INTER_CUBIC)

        # Normalize
        lr_input = lr_upscaled.astype('float32') / 255.0
        hr_target = hr.astype('float32') / 255.0

        return np.expand_dims(lr_input, axis=0), np.expand_dims(hr_target, axis=0)


### Validation and Evaluation

In [23]:
val_gen = FullImageValidationGen(
    lr_dir="DIV2K/valid_LR_X4/DIV2K_valid_LR_bicubic/X4",
    hr_dir="DIV2K/valid_HR/DIV2K_valid_HR"
)

mse_total, mae_total = 0, 0

for i in range(len(val_gen)):
    lr_img, hr_img = val_gen[i]
    pred = model.predict(lr_img)

    mse = tf.reduce_mean(tf.square(pred - hr_img)).numpy()
    mae = tf.reduce_mean(tf.abs(pred - hr_img)).numpy()

    mse_total += mse
    mae_total += mae

mse_avg = mse_total / len(val_gen)
mae_avg = mae_total / len(val_gen)

print(f"\n✅ Full-image Validation")
print(f"MSE:  {mse_avg:.6f}")
print(f"MAE:  {mae_avg:.6f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 26s 26s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 25s 25s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 30s 30s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 24s 24s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 32s 32s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 28s 28s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 23s 23s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 28s 28s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 32s 32s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 39s 39s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 28s 28s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 33s 33s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 30s 30s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 28s 28s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 32s 32s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 29s 29s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 33s 33s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 41s 41s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 33s 33s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 29s 29s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 29s 29s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 36s 36s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 29s 29s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 31s 31s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 30s 30s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 36s 36s/step
1/1 ━━━━━━━━